In [6]:
from data_loader import Data_set_body
import torch, utils, trimesh, pymesh, os
from model.model import EncoderDecoder
from IPython.display import Image

option = utils.Option()
train_Data = Data_set_body(option.number_points, 'overfit', "headposes")
if torch.cuda.is_available():
    option.device = torch.device(f"cuda:0")
else:
    option.device = torch.device(f"cpu")

model = EncoderDecoder(option)
model_dict_file = 'runs/model_best_headposes3.ckpt'
model.load_state_dict(torch.load(model_dict_file))
ind_0 = 0
ind_1 = 26

id_0 = ind_0 if ind_0>=0 else len(train_Data)+ind_0
input_0 = train_Data[ind_0]['img'].unsqueeze(0).float().to(option.device)
id_1 = ind_1 if ind_1>=0 else len(train_Data)+ind_1
input_1 = train_Data[ind_1]['img'].unsqueeze(0).float().to(option.device)
model.eval()
lat_code_0 = model.encoder(input_0)
lat_code_1 = model.encoder(input_1)
mesh0 = model.generate_mesh(input_0)
mesh1 = model.generate_mesh(input_1)


New MLP decoder : hidden size 512, num_layers 2, activation relu
40962


In [7]:
# utils.k3d_visualize_mesh(mesh0, flip_axes=False)
print(lat_code_0)

tensor([[-0.0780, -0.6468, -1.3556,  ..., -0.2044,  0.2780,  1.5939]],
       device='cuda:0', grad_fn=<AddmmBackward>)


In [9]:
exp_time = "28jun_2"
if not os.path.exists(f"runs/for_gif/{exp_time}_exp"):
    os.makedirs(f"runs/for_gif/{exp_time}_exp")
num_interpolation = 60
for i in range(num_interpolation):
    latent_new = lat_code_0 + i * (lat_code_1 - lat_code_0) / num_interpolation
    mesh_new = model.generate_mesh_by_latent(latent_new)
    pymesh.save_mesh(f"runs/for_gif/{exp_time}_exp/mesh_human_head_{i}.obj", mesh_new, ascii=True)


In [10]:
mesh_folder = f"runs/for_gif/{exp_time}_exp"

utils.meshes_to_gif(mesh_folder, f"runs/for_gif/{exp_time}_exp.gif", 10)

visualizing: 100%|██████████| 60/60 [00:25<00:00,  2.35it/s]
